In [1]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
import re
from itertools import islice
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading the JSON file into a dataframe
original_df = pd.read_json('ol_cdump.json',lines=True)

#Checking the head of the dataframe
original_df.head()

,latest_revision,revision,title,languages,subjects,publish_country,by_statement,type,location,other_titles,...,contributors,purchase_url,fuller_name,website,isbn_invalid,ia_loaded_id,isbn_odd_length,ia_box_id,full_title,excerpts
0,2,2,The effect of differentiated marking tools and...,[{'key': '/languages/eng'}],"[Creative thinking -- Testing, Educational psy...",gau,by Lillian Rose Arnold,{'key': '/type/edition'},[NBuC],"[Marking tools, Motivational treatment]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,Comparison of the nominal grouping and sequenc...,[{'key': '/languages/eng'}],"[Problem solving, Group problem solving, Socia...",lau,by Walter Edward Stead,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,Professional accident investigation,[{'key': '/languages/eng'}],"[Industrial accidents -- Investigation, Indust...",gau,by Raymond L. Kuhlman,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,2,I chauceriani scozzesi,[{'key': '/languages/ita'}],"[English poetry -- Scottish authors, Scottish ...",it,a cura di Sergio Rossi,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2,Lezioni zurighesi sul Petrarca,[{'key': '/languages/ita'}],"[Petrarca, Francesco, -- 1304-1374]",it,Francesco de Sanctis ; a cura di Sergio Romagnoli,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148163 entries, 0 to 148162
Data columns (total 77 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   latest_revision      148163 non-null  int64  
 1   revision             148163 non-null  int64  
 2   title                146852 non-null  object 
 3   languages            140960 non-null  object 
 4   subjects             97613 non-null   object 
 5   publish_country      132102 non-null  object 
 6   by_statement         106091 non-null  object 
 7   type                 148163 non-null  object 
 8   location             17401 non-null   object 
 9   other_titles         25664 non-null   object 
 10  publishers           136729 non-null  object 
 11  last_modified        148163 non-null  object 
 12  key                  148163 non-null  object 
 13  authors              145622 non-null  object 
 14  publish_places       135581 non-null  object 
 15  oclc_number      

In [4]:
original_df.columns

Index(['latest_revision', 'revision', 'title', 'languages', 'subjects',
       'publish_country', 'by_statement', 'type', 'location', 'other_titles',
       'publishers', 'last_modified', 'key', 'authors', 'publish_places',
       'oclc_number', 'pagination', 'created', 'notes', 'number_of_pages',
       'publish_date', 'works', 'subtitle', 'series', 'contributions',
       'genres', 'table_of_contents', 'lc_classifications', 'subject_places',
       'first_publish_date', 'edition_name', 'isbn_10', 'dewey_decimal_class',
       'lccn', 'identifiers', 'weight', 'physical_format', 'covers', 'isbn_13',
       'physical_dimensions', 'uri_descriptions', 'uris', 'url',
       'source_records', 'first_sentence', 'work_titles', 'description',
       'subject_times', 'ocaid', 'download_url', 'dewey_number', 'name',
       'subject_place', 'subject_people', 'personal_name', 'alternate_names',
       'photos', 'oclc_numbers', 'copyright_date', 'classifications',
       'birth_date', 'bio', 'links

In [34]:
import pandas_profiling
   
  
# run the profile report
profile = original_df.sample(1000).profile_report(title='Pandas Profiling Report')
   
# save the report as html file
profile.to_file(output_file="pandas_profiling1.html")

Summarize dataset:   0%|          | 0/92 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
#calculating the total number of rows in raw data
print("Total number of rows in raw data: "+str(original_df.shape[0]))

Total number of rows in raw data: 148163


In [6]:
# calculate null percentage
percent_missing = original_df.isnull().sum() * 100 / len(original_df)
missing_value_df = pd.DataFrame({'column_name': original_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values(by='percent_missing', inplace=True)
missing_value_df.loc[['title','publish_date','number_of_pages','authors']]

,column_name,percent_missing
title,title,0.884836
publish_date,publish_date,6.705453
number_of_pages,number_of_pages,22.321362
authors,authors,1.715003


In [7]:
#calculating the mean,min,max,etc for numerical columns
original_df.describe()

,latest_revision,revision,number_of_pages
count,148163.000000,148163.000000,115091.000000
mean,2.255691,2.255691,222.659991
std,0.755319,0.755319,291.893138
min,1.000000,1.000000,0.000000
25%,2.000000,2.000000,87.000000
50%,2.000000,2.000000,189.000000
75%,2.000000,2.000000,295.000000
max,50.000000,50.000000,48418.000000


In [8]:
#Assigning dataframe to a new dataframe for ease of use
reference_df=original_df

#Number of null values in 'publish_date' column 
print("Number of null values in 'publish_date' column: "+ str(reference_df['publish_date'].isna().sum()))

#Eliminating data having null values in 'publish_date' column
reference_df=reference_df[reference_df['publish_date'].notna()]

#Number of null values in 'publish_date' and no of rows left
print("Number of rows removed from 'publish_date' column: "+ str(original_df.shape[0]-reference_df.shape[0]))
print("Number of rows left:"+str(reference_df.shape[0]))


Number of null values in 'publish_date' column: 9935
Number of rows removed from 'publish_date' column: 9935
Number of rows left:138228


In [9]:
reference_df['publish_date'].unique()

array(['1975', '1976', '1977', ..., 'April 5, 1993', 'December 1968',
       'July 15, 1989'], dtype=object)

In [36]:
#Checking for alphanumeric values in publish_date
reference_df['publish_date_numeric_or_not']=reference_df['publish_date'].apply(utils.check_alpha_numeric)
reference_df['publish_date_numeric_or_not'].unique()

#Count for numeric
print("No of numeric values in 'publish_date': "+ str(reference_df['publish_date_numeric_or_not'].value_counts()[0]))
print("No of alphanumeric values in 'publish_date': "+ str(reference_df['publish_date_numeric_or_not'].value_counts()[1]))

## get percentage of numeric and alpha numeric
percentage_numeric=reference_df['publish_date_numeric_or_not'].value_counts()[False]
print("numeric percentage is {0}".format(percentage_numeric*100/reference_df.shape[0]))
percentage_alpha_numeric=reference_df['publish_date_numeric_or_not'].value_counts()[True]
print("alpha numeric percentage is {0}".format(percentage_alpha_numeric*100/reference_df.shape[0]))


No of numeric values in 'publish_date': 75429
No of alphanumeric values in 'publish_date': 1993
numeric percentage is 97.42579628529359
alpha numeric percentage is 2.5742037147064143


In [11]:
reference_df.head()

,latest_revision,revision,title,languages,subjects,publish_country,by_statement,type,location,other_titles,...,purchase_url,fuller_name,website,isbn_invalid,ia_loaded_id,isbn_odd_length,ia_box_id,full_title,excerpts,publish_date_numeric_or_not
0,2,2,The effect of differentiated marking tools and...,[{'key': '/languages/eng'}],"[Creative thinking -- Testing, Educational psy...",gau,by Lillian Rose Arnold,{'key': '/type/edition'},[NBuC],"[Marking tools, Motivational treatment]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2,2,Comparison of the nominal grouping and sequenc...,[{'key': '/languages/eng'}],"[Problem solving, Group problem solving, Socia...",lau,by Walter Edward Stead,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2,2,Professional accident investigation,[{'key': '/languages/eng'}],"[Industrial accidents -- Investigation, Indust...",gau,by Raymond L. Kuhlman,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2,2,I chauceriani scozzesi,[{'key': '/languages/ita'}],"[English poetry -- Scottish authors, Scottish ...",it,a cura di Sergio Rossi,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2,2,Lezioni zurighesi sul Petrarca,[{'key': '/languages/ita'}],"[Petrarca, Francesco, -- 1304-1374]",it,Francesco de Sanctis ; a cura di Sergio Romagnoli,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [12]:
#To get the year column
reference_df['pub_year'] = reference_df['publish_date'].apply(utils.check_get_year)

#To get the dataframe with non-null publish year values
reference_df = reference_df[reference_df['pub_year'].notna()]

#Number of rows left
print("Number of rows removed from 'publish_date' column: "+ str(original_df.shape[0]-reference_df.shape[0]))
print("Number of rows left:"+str(reference_df.shape[0]))

Number of rows removed from 'publish_date' column: 10433
Number of rows left:137730


In [13]:
reference_df

,latest_revision,revision,title,languages,subjects,publish_country,by_statement,type,location,other_titles,...,fuller_name,website,isbn_invalid,ia_loaded_id,isbn_odd_length,ia_box_id,full_title,excerpts,publish_date_numeric_or_not,pub_year
0,2,2,The effect of differentiated marking tools and...,[{'key': '/languages/eng'}],"[Creative thinking -- Testing, Educational psy...",gau,by Lillian Rose Arnold,{'key': '/type/edition'},[NBuC],"[Marking tools, Motivational treatment]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1975
1,2,2,Comparison of the nominal grouping and sequenc...,[{'key': '/languages/eng'}],"[Problem solving, Group problem solving, Socia...",lau,by Walter Edward Stead,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1976
2,2,2,Professional accident investigation,[{'key': '/languages/eng'}],"[Industrial accidents -- Investigation, Indust...",gau,by Raymond L. Kuhlman,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1977
3,2,2,I chauceriani scozzesi,[{'key': '/languages/ita'}],"[English poetry -- Scottish authors, Scottish ...",it,a cura di Sergio Rossi,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1964
4,2,2,Lezioni zurighesi sul Petrarca,[{'key': '/languages/ita'}],"[Petrarca, Francesco, -- 1304-1374]",it,Francesco de Sanctis ; a cura di Sergio Romagnoli,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148158,2,2,Duzakhi-ha,[{'key': '/languages/per'}],NaN,ir,Parviz Qazi Said.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2003
148159,3,3,Visitor survey,[{'key': '/languages/eng'}],"[San Francisco Zoo (San Francisco, Calif.), Zo...",cau,prepared and submitted by Esherick Homsey Dodg...,{'key': '/type/edition'},NaN,[Plan for the San Francisco Zoological Gardens...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1976
148160,2,2,Lavsun dar ashiyanih yi marg,[{'key': '/languages/per'}],NaN,ir,Parviz Qazi Said.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2001
148161,2,2,Fun with Chinese horoscopes,[{'key': '/languages/eng'}],[Horoscopes.],si,written and illustrated by Evelyn Lip.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1987


In [14]:
#To get the publishing month column
reference_df['pub_month'] = reference_df['publish_date'].apply(utils.check_get_month)

#To check if any null values present in month column
print("Null values in month column:"+str(reference_df['pub_month'].isna().sum()))

#Convert year into integer
reference_df['pub_year']=reference_df['pub_year'].astype(int)


Null values in month column:0


In [16]:
reference_df

,latest_revision,revision,title,languages,subjects,publish_country,by_statement,type,location,other_titles,...,website,isbn_invalid,ia_loaded_id,isbn_odd_length,ia_box_id,full_title,excerpts,publish_date_numeric_or_not,pub_year,pub_month
0,2,2,The effect of differentiated marking tools and...,[{'key': '/languages/eng'}],"[Creative thinking -- Testing, Educational psy...",gau,by Lillian Rose Arnold,{'key': '/type/edition'},[NBuC],"[Marking tools, Motivational treatment]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1975,fir
1,2,2,Comparison of the nominal grouping and sequenc...,[{'key': '/languages/eng'}],"[Problem solving, Group problem solving, Socia...",lau,by Walter Edward Stead,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1976,fir
2,2,2,Professional accident investigation,[{'key': '/languages/eng'}],"[Industrial accidents -- Investigation, Indust...",gau,by Raymond L. Kuhlman,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1977,fir
3,2,2,I chauceriani scozzesi,[{'key': '/languages/ita'}],"[English poetry -- Scottish authors, Scottish ...",it,a cura di Sergio Rossi,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1964,fir
4,2,2,Lezioni zurighesi sul Petrarca,[{'key': '/languages/ita'}],"[Petrarca, Francesco, -- 1304-1374]",it,Francesco de Sanctis ; a cura di Sergio Romagnoli,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1955,fir
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148158,2,2,Duzakhi-ha,[{'key': '/languages/per'}],NaN,ir,Parviz Qazi Said.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2003,fir
148159,3,3,Visitor survey,[{'key': '/languages/eng'}],"[San Francisco Zoo (San Francisco, Calif.), Zo...",cau,prepared and submitted by Esherick Homsey Dodg...,{'key': '/type/edition'},NaN,[Plan for the San Francisco Zoological Gardens...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1976,fir
148160,2,2,Lavsun dar ashiyanih yi marg,[{'key': '/languages/per'}],NaN,ir,Parviz Qazi Said.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2001,fir
148161,2,2,Fun with Chinese horoscopes,[{'key': '/languages/eng'}],[Horoscopes.],si,written and illustrated by Evelyn Lip.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1987,fir


In [38]:
#Number of rows left after removing null values of pub_year & pub_month column
print("Number of rows left:"+str(reference_df.shape[0]))

Number of rows left:77422


In [39]:
#Filter condition
reference_df=reference_df.loc[(reference_df['title'].notna()) & (reference_df['number_of_pages']>20) & (reference_df['pub_year']>1950) & (reference_df['pub_year']<=2021)]

#Number of rows left after removing null values of pub_year & pub_month column
print("Number of rows left after data cleaning & filtering conditions:"+str(reference_df.shape[0]))

Number of rows left after data cleaning & filtering conditions:77422


In [40]:
#To check null values count
print("null values count: "+str(reference_df['genres'].isna().sum()))

#To unpack genre column into a single large list
genre_list=[]
for genre in reference_df['genres']:
    if not genre is np.nan:
        genre_list=genre_list+genre

null values count: 60936


In [42]:
#Book with the most pages
reference_df[reference_df['number_of_pages']==reference_df['number_of_pages'].max()]
#print("Book with most pages: "+str(reference_df.groupby('title')['number_of_pages'].sum().reset_index().max()['title']))

,latest_revision,revision,title,languages,subjects,publish_country,by_statement,type,location,other_titles,...,website,isbn_invalid,ia_loaded_id,isbn_odd_length,ia_box_id,full_title,excerpts,publish_date_numeric_or_not,pub_year,pub_month
51907,2,2,Nihon shokuminchi kenchikuron,[{'key': '/languages/jpn'}],[Architecture -- Japan -- Colonies -- History....,ja,Nishizawa Yasuhiko cho.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2008,fir


In [19]:
#To get top 5 genre with most books
d = {x:genre_list.count(x) for x in genre_list}
sorted_dict=dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
dfx= pd.DataFrame(sorted_dict.values(),sorted_dict)
dfx.head(5)

,0
Fiction.,3301
Biography.,2360
Juvenile literature.,1538
Exhibitions.,836
Juvenile fiction.,525


In [20]:
#To create a filtered dataframe having publish years between 1950 & 1970
df_filtered_1970=reference_df[reference_df['pub_year']<=1970]



In [21]:
df_filtered_1970.columns

Index(['latest_revision', 'revision', 'title', 'languages', 'subjects',
       'publish_country', 'by_statement', 'type', 'location', 'other_titles',
       'publishers', 'last_modified', 'key', 'authors', 'publish_places',
       'oclc_number', 'pagination', 'created', 'notes', 'number_of_pages',
       'publish_date', 'works', 'subtitle', 'series', 'contributions',
       'genres', 'table_of_contents', 'lc_classifications', 'subject_places',
       'first_publish_date', 'edition_name', 'isbn_10', 'dewey_decimal_class',
       'lccn', 'identifiers', 'weight', 'physical_format', 'covers', 'isbn_13',
       'physical_dimensions', 'uri_descriptions', 'uris', 'url',
       'source_records', 'first_sentence', 'work_titles', 'description',
       'subject_times', 'ocaid', 'download_url', 'dewey_number', 'name',
       'subject_place', 'subject_people', 'personal_name', 'alternate_names',
       'photos', 'oclc_numbers', 'copyright_date', 'classifications',
       'birth_date', 'bio', 'links

In [26]:
#Number of null values in authors column
print("Number of null values in authors column: "+str(df_filtered_1970['authors'].isna().sum()))

#Filtering out null values from authors column
df_filtered_1970=df_filtered_1970[df_filtered_1970['authors'].notna()]

#Number of rows left following filtering it
print("Number of rows left: "+str(df_filtered_1970.shape[0]))

Number of null values in authors column: 0
Number of rows left: 19558


In [28]:
#Creating a new column author ID
df_filtered_1970['author_ID']=df_filtered_1970['authors'].apply(lambda x:str(x).split("/")[2][0:-3])

In [29]:
df_filtered_1970

,latest_revision,revision,title,languages,subjects,publish_country,by_statement,type,location,other_titles,...,isbn_invalid,ia_loaded_id,isbn_odd_length,ia_box_id,full_title,excerpts,publish_date_numeric_or_not,pub_year,pub_month,author_ID
3,2,2,I chauceriani scozzesi,[{'key': '/languages/ita'}],"[English poetry -- Scottish authors, Scottish ...",it,a cura di Sergio Rossi,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1964,fir,OL5900510A
4,2,2,Lezioni zurighesi sul Petrarca,[{'key': '/languages/ita'}],"[Petrarca, Francesco, -- 1304-1374]",it,Francesco de Sanctis ; a cura di Sergio Romagnoli,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1955,fir,OL135548A
5,2,2,El saber ginecologico del padre Feijoo,[{'key': '/languages/spa'}],"[Feijóo y Montenegro, Benito Jerónimo, -- 1676...",sp,E. Junceda Avello,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1964,fir,OL6535899A
8,2,2,Reflex French,[{'key': '/languages/eng'}],[French language -- Grammar],fr,"by E. J. H. Greene, Manoël Faucher, Dennis M. ...",{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1966,fir,OL133670A
13,2,2,Adventuring with books,[{'key': '/languages/eng'}],"[Library science -- Outlines, syllabi, etc, Re...",nyu,NaN,{'key': '/type/edition'},[NBuC],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1961,fir,OL6533579A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148096,3,3,Ferries of the South,[{'key': '/languages/eng'}],[Ferries -- California],xx,by Walt Wheelock ; illustrated by the author.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1964,fir,OL409560A
148131,2,2,Anglo-ukraïnsʹkyĭ slovnyk (shkilʹnyĭ),[{'key': '/languages/ukr'}],[English language -- Dictionaries -- Ukrainian...,un,NaN,{'key': '/type/edition'},NaN,[English-Ukrainian dictionary.],...,NaN,NaN,NaN,NaN,NaN,NaN,False,1955,fir,OL4673667A
148132,2,2,English-Ukrainian dictionary,[{'key': '/languages/eng'}],[English language -- Dictionaries -- Ukrainian.],nyu,compiled by M. L. Podvesko.,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1963,fir,OL4673667A
148135,3,3,Englesko-srpskohrvatski rec̆nik.,[{'key': '/languages/mul'}],[English language -- Dictionaries -- Serbo-Cro...,xx,NaN,{'key': '/type/edition'},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1959,fir,OL367501A


In [33]:
#To find number of authors and number of books published per month for years between 1950 and 1970
df_filtered_1970.groupby(['pub_year','pub_month'])[['author_ID','title']].nunique()

author_ID  title
pub_year pub_month                  
1951     fir              648    671
1952     fir              601    632
1953     fir              600    618
         oct                1      1
1954     fir              644    657
         jun                1      1
1955     fir              588    607
         jun                1      1
1956     fir              638    654
1957     fir              705    734
         sept               1      1
1958     fir              753    785
         jun                1      1
1959     fir              830    872
1960     fir              921    955
         jan                1      1
         jun                2      2
1961     fir             1026   1064
         oct                1      1
1962     dec                1      1
         fir             1064   1112
         sept               1      1
1963     fir             1099   1166
         jul                1      1
1964     fir             1194   1255
         jun                1      1
         may                1      1
1965     fir             1248   1319
         jun                1      1
1966     fir             1203   1259
         nov                1      1
1967     fir             1243   1308
         jun                1      1
1968     fir             1082   1140
         jul                1      1
         jun                3      3
         may                1      1
1969     fir             1208   1264
         jan                1      1
         jun                1      1
1970     fir             1139   1211
         jun                2      2